# fotd-interactive-football-pitch
Inspired by the latest Friends of Tracking Data episode [How Tracking Data is Used in Football and What are the Future Challenges](https://www.youtube.com/watch?v=kHTq9cwdkGA), I want to share how easy it can be to create interactivate widgets for tracking data in Python. (If you know all the libraries!).
Jupyter widgets allow to add interactivity to Jupyter notebooks. This repository contains an example notebook which uses `qgrid` and `bqplot` to create a simple interactive football pitch.  
Important: The sample data was downloaded from [https://github.com/metrica-sports/sample-data](https://github.com/metrica-sports/sample-data). Please visit this repository in case you want to reuse it. 
The notebook presents three different use-cases of Jupyter widgets to animate and work with tracking data.

## Use-cases
- Use-case 1: Moving Players on a Football Pitch
- Use-case 2: Animate Tracking Data
- Use-case 3: Replay Events with Tracking Data

In [1]:
import os
import ipywidgets as widgets

from bqplot import *
import numpy as np
import pandas as pd
import qgrid

In [2]:
# X = [-55.5, 55.5]
# Y = [-37.0, 37.0]  

# Map the pitch coordinates to [0,0] on the top left and [1,1] on bottom right.
PITCH_WIDTH = 105.0
PITCH_HEIGHT = 68.0
OFFSET = 3.0
OFFSET_WIDTH = OFFSET/PITCH_WIDTH
OFFSET_HEIGHT = OFFSET/PITCH_HEIGHT
X = [-OFFSET_WIDTH, 1+OFFSET_WIDTH]
Y_rev = [-OFFSET_HEIGHT, 1+OFFSET_HEIGHT]

# Only used to fix some issues in voila
WIDTH = 506.7
HEIGHT = 346.7
FACTOR = 1.8

class RadarViewWidget(widgets.VBox):
    def __init__(self, pitch_img='pitch.png', enable_logging=True):
        super().__init__()
        self.pitch_img = pitch_img
        self.enable_logging = enable_logging
        self.image = self.__init_image()
        self.home_scatter = self.__init_scatter()
        self.away_scatter = self.__init_scatter()
        self.ball_scatter = self.__init_scatter(size=64, selected_opacity=1.0)
        
        self.fig = Figure(marks=[self.image, self.home_scatter, self.away_scatter, self.ball_scatter], padding_x=0, padding_y=0)
        self.fig.layout = widgets.Layout(width=f'{WIDTH*FACTOR}px', height=f'{HEIGHT*FACTOR}px')
        self.output = widgets.Output()
        
        self.children = [self.fig, self.output]
        
    
    def __init_image(self):
        # read pitch image
        image_path = os.path.abspath(self.pitch_img)

        with open(image_path, 'rb') as f:
            raw_image = f.read()
        ipyimage = widgets.Image(value=raw_image, format='png')

        scales_image = {'x': LinearScale(), 'y': LinearScale(reverse=True)}
        axes_options = {'x': {'visible': True}, 'y': {'visible': True}}

        image = Image(image=ipyimage, scales=scales_image, axes_options=axes_options)
        # Full screen
        image.x = X
        image.y = Y_rev
        
        return image
        
    def __init_scatter(self, size=128, selected_opacity=0.6, unselected_opacity=1.0):
        scales={'x': LinearScale(min=X[0], max=X[1]), 'y': LinearScale(min=Y_rev[0], max=Y_rev[1], reverse=True)}
        axes_options = {'x': {'visible': False}, 'y': {'visible': False}}

        scatter = Scatter(
                            scales= scales, 
                            default_size=size,
                            interactions={'click': 'select'},
                            selected_style={'opacity': selected_opacity, 'stroke': 'Black'},
                            unselected_style={'opacity': unselected_opacity},
                            axes_options=axes_options)
        scatter.enable_move = True
        
        if self.enable_logging:
            scatter.on_drag_end(self.output_data)
        
        return scatter
    
    def disable_move(self):
        self.home_scatter.enable_move = False
        self.away_scatter.enable_move = False
        self.ball_scatter.enable_move = False

    def output_data(self, name, data):
        new_x = round(data['point']['x'], 2)
        new_y = round(data['point']['y'], 2)
        
        self.output.clear_output()
        with self.output:
            print(f'Changed player coordinates to ({new_x},{new_y})')
    
    def set_data(self, frameset):
        self.home_scatter.x = frameset['home_x']
        self.home_scatter.y = frameset['home_y']
        self.home_scatter.names=frameset['home_names'],
        self.home_scatter.colors=frameset['home_color']
        
        self.away_scatter.x = frameset['away_x']
        self.away_scatter.y = frameset['away_y']
        self.away_scatter.names=frameset['away_names'],
        self.away_scatter.colors=frameset['away_color']
        
        self.ball_scatter.x = frameset['ball_x']
        self.ball_scatter.y = frameset['ball_y']
        self.ball_scatter.colors = frameset['ball_color']
        


In [9]:
def get_playerid(name): return name[6:]
def get_frameset(frameid):
    return {
        'home_x': df_home.iloc[frameid, 2:-3:2].dropna().values,
        'home_y': df_home.iloc[frameid, 3:-3:2].dropna().values,
        'home_names': [get_playerid(name) for name in df_home.iloc[frameid, 2:-3:2].dropna().index.values],
        'home_color': ['blue'],
        'ball_x': df_home.iloc[frameid, [-2]].values,
        'ball_y': df_home.iloc[frameid, [-1]].values,
        'ball_color': ['black'],
        'away_x': df_away.iloc[frameid, 2:-3:2].dropna().values,
        'away_y': df_away.iloc[frameid, 3:-3:2].dropna().values,
        'away_names': [get_playerid(name) for name in df_away.iloc[frameid, 2:-3:2].dropna().index.values],
        'away_color': ['red'],
        
    }


## Use-case 1: Moving Players on a Football Pitch
Here, you see how you can use `bqplot` to allow players and the ball on a pitch to be movable. 

In [10]:
widget = RadarViewWidget()
display(widget)

df_home = pd.read_csv('metrica_sample_data/Sample_Game_1_RawTrackingData_Home_Team.csv', index_col=0, skiprows=[0,1])
df_away = pd.read_csv('metrica_sample_data/Sample_Game_1_RawTrackingData_Away_Team.csv', index_col=0, skiprows=[0,1])

frameid = 0
initial_frameset = get_frameset(frameid)

widget.set_data(initial_frameset)

RadarViewWidget(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(…

Move any player or the ball on the pitch and get the information where you moved the scatter point by observing changes with `on_drag_end`. You might want to use this functionality to interactively test the output of your models like `expected-goals`.  
We disable this functionalty for the other use-cases.

In [7]:
widget.disable_move()

## Use-case 2: Animate tracking data
You can also hook up different widgets to interact with each other. As an example, we allow to replay the tracking data.

In [11]:
# Speedup animation
STEP = 2

display(widget)

play = widgets.Play(
    value=0,
    step=STEP,
    max=len(df_home),
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider(max=len(df_home))
widgets.jslink((play, 'value'), (slider, 'value'))

def change_data(change):
    widget.set_data(get_frameset(change['new']))
    
slider.observe(change_data, names='value')
slider.value
widgets.HBox([play, slider])

RadarViewWidget(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(…

Now, clicking play or using the slider will update the pitch. But, you can also do this programmatically.

In [12]:
slider.value = 24

## Use-case 3: Replay Events with Tracking Data
Because the tracking data and the events are actually synchronised, we can build a simple widget that allows to jump to the event in the tracking data and replay it using our previous widgets and `qgrid` for an interactive table for the events. 

In [13]:
events = pd.read_csv('metrica_sample_data/Sample_Game_1_RawEventsData.csv')

In [14]:
events.tail()

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
1740,Home,PASS,NaN,2,143361,5734.44,143483,5739.32,Player12,Player13,0.60,0.33,0.19,0.95
1741,Home,PASS,NaN,2,143578,5743.12,143593,5743.72,Player13,Player4,0.09,0.88,0.14,0.69
1742,Home,BALL LOST,INTERCEPTION,2,143598,5743.92,143618,5744.72,Player4,NaN,0.13,0.69,0.07,0.61
1743,Away,RECOVERY,BLOCKED,2,143617,5744.68,143617,5744.68,Player16,NaN,0.05,0.62,NaN,NaN
1744,Away,BALL OUT,NaN,2,143622,5744.88,143630,5745.20,Player16,NaN,0.05,0.63,0.03,1.01


In [10]:
COLUMNS = ['Type','Team','Start Frame','Start Time [s]','Start X','Start Y']
events_short = events[COLUMNS].copy()

In [12]:
# Define qgrid widget
qgrid.set_grid_option('maxVisibleRows', 10)
col_opts = { 
    'editable': False,
}

def on_row_selected(change):
    """callback for row selection: update selected points in scatter plot"""
    filtered_df = qgrid_widget.get_changed_df() 
    event = filtered_df.iloc[change.new]
    widget.set_data(get_frameset((int(event['Start Frame'].item()))))
    slider.value = int(event['Start Frame'].item())
   
        
qgrid_widget = qgrid.show_grid(events_short, show_toolbar=False, column_options=col_opts)
qgrid_widget.layout = widgets.Layout(width='920px')
   
qgrid_widget.observe(on_row_selected, names=['_selected_rows'])

display(widget)
display(widgets.HBox([play, slider]))
display(qgrid_widget)

RadarViewWidget(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(…

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Select a row to jump to the specific event in the tracking data. 
You can easily filter for events and teams clicking on the filter symbol in the column headers.

## Next
If you want to add some functionality, here are some ideas:
- Add the current game time to the pitch plot (and update it when playing).
- Add some functionality like voronoi cells or convex hulls of teams as implemented [here](https://github.com/seidlr/Game-Animation).
- Add a auto-replay option: When selecting an event, the animation should start at this event and stop of it is over as documented in the columns `End X` and `End Y`.
- Create an overlay/tooltip for player information using `bqplot.tooltip`.
- Calculate the distance covered for each player and the ball in the match.